In [1]:
import pandas as pd
import random
from caboose_nbr.pernir import Pernir
from caboose_nbr.evaluate_recommendation import evaluate

In [2]:
train_baskets = pd.read_csv('data/instacart_30k/train_baskets.csv.gz')
test_samples = pd.read_csv('data/instacart_30k/test_samples.csv')

In [3]:
all_users = train_baskets['user_id'].tolist()
sample_users = random.sample(all_users,1000)

In [4]:
train_baskets = train_baskets[train_baskets['user_id'].isin(sample_users)]
test_samples = test_samples[test_samples['user_id'].isin(sample_users)]
test_samples = test_samples.groupby('user_id').sample(1)

In [5]:
print(train_baskets.shape)
print(test_samples.shape)
print(train_baskets[['user_id','item_id']].drop_duplicates().shape)

(400860, 10)
(958, 5)
(120659, 2)


In [6]:
pernir_similaripy = Pernir(train_baskets, test_samples,'similaripy')
pernir_similaripy.train()

start of knn


Done: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 958/958 [00:00<00:00, 15223.76it/s]


knn finished


In [7]:
pernir_similaripy_preds , sim_scores = pernir_similaripy.predict()

num test users: 958
0 samples passed
500 samples passed


In [8]:
pernir_caboose = Pernir(train_baskets, test_samples,'caboose')
pernir_caboose.train()

start of knn
knn finished


--Creating transpose of R...
--Computing row norms...
--Configuring for top-k -- num_threads: 4; pinning? false;
--Scheduling parallel top-k computation...


In [9]:
pernir_caboose_preds , caboose_scores = pernir_caboose.predict()

num test users: 958
0 samples passed
500 samples passed


In [15]:
test_users = test_samples['user_id'].tolist()
bad_users  = []
for user in test_users:
    if pernir_similaripy.user_neighbors[user] != pernir_caboose.user_neighbors[user]:
        bad_users.append(user)
print(len(bad_users))

958


In [10]:
bad_preds  = []
for i in range(len(pernir_similaripy_preds)):
    if pernir_similaripy_preds[i] != pernir_caboose_preds[i]:
        bad_preds.append(i)
print(len(bad_preds))

684


In [11]:
bad_preds[:10]

[1, 2, 3, 4, 5, 6, 8, 9, 10, 11]